In [1]:
!pip install -qqq langchain_openai
!pip install -qqq -U langchain-community
!pip install -qqq -U langchain_chroma
!pip install -qqq -U langchain-huggingface
!pip install -qqq -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.2/580.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 21.5 MB/s eta 0:00:00


In [1]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
# !pip install --upgrade --force-reinstall pandas datasets

In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("cnamuangtoun/resume-job-description-fit")

train.csv:   0%|          | 0.00/53.4M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1759 [00:00<?, ? examples/s]

In [4]:
dataset.column_names

{'train': ['resume_text', 'job_description_text', 'label'],
 'test': ['resume_text', 'job_description_text', 'label']}

In [5]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [9]:
dataset['train'][0]

{'resume_text': "SummaryHighly motivated Sales Associate with extensive customer service and sales experience. Outgoing sales professional with track record of driving increased sales, improving buying experience and elevating company profile with target market.\nHighlights-Soft Skills: Public Speaking, Public Relations, Team Building, Project Management, Procedure writing, Staff Supervision and Management, Ability to interface with professionals on all levels. Accomplishments, Honors, and Activities -Board of Directors Member for the Food Bank of Corpus Christi from November 2010 to April 2013. -Held Life Insurance License -Basketball Official (Referee) High School Varsity Level.\nExperienceAccountant,08/2014-05/2015Aspirus–Owen,WI,Perform daily and routine accounting functions for two main companies and five small royalty companies. Responsibilities include but are not limited to the following: Accounts Payable, Accounts Receivable, Manage and reconcile funds for multiple banks accou

In [7]:
print(len(dataset['train']))
print(len(dataset['test']))

6241
1759


In [8]:
db_name = "job_embeddings_db"
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en",encode_kwargs={'normalize_embeddings':True})

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
import os

if os.path.exists(db_name):
  Chroma(persist_directory = db_name,embedding_function = embeddings).delete_collection()

In [11]:
# Initialize the Chroma vector store
vector_store = Chroma(persist_directory=db_name, embedding_function=embeddings)

# Fetch and store embeddings
for record in dataset['train']:
    resume_text = record['resume_text']
    job_description_text = record['job_description_text']

    # Generate embeddings for resume and job description
    resume_embedding = embeddings.embed_query(resume_text)
    job_description_embedding = embeddings.embed_query(job_description_text)

    # Store embeddings in the vector store with metadata
    vector_store.add_texts(
        texts=[resume_text, job_description_text],
        metadatas=[{'type': 'resume'}, {'type': 'job_description'}]
    )

# Persist the vector store
# vector_store.persist()

In [12]:
embedding_sample = embeddings.embed_query("This is a sample text.")
print(len(embedding_sample))

384


In [14]:
from langchain_openai import ChatOpenAI

from google.colab import userdata
openai_api = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(api_key=openai_api)

In [15]:
retriever = vector_store.as_retriever()

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt_search_query = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Based on the above conversation, generate a structured search query to find relevant job postings. "
             "Ensure the query includes key skills, experience level, and location from the user's request.")
])

In [17]:
from langchain.chains import create_history_aware_retriever

retriever_chain = create_history_aware_retriever(llm, retriever, prompt_search_query)

In [18]:
prompt_get_answer = ChatPromptTemplate.from_messages([
    ("system", "You are an AI-powered recruitment assistant. Your goal is to provide job recommendations "
               "based on the user's skills, experience, and location. Use the following job postings "
               "to generate a professional and structured response. Do not return context and system messages.\n\n"
               "{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt_get_answer)

In [20]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [23]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
response = retrieval_chain.invoke({
"chat_history":chat_history,
"input":"I am skilled in Machine learning and Data science with 3 years of experience. Looking for a job in San Francisco Location. Can you provide me 5 options?"
})
print(response["answer"])

Based on your skills and experience in Machine learning and Data science, here are some job recommendations in the San Francisco location:

1. Role: Data Scientist
   Company: BayAreaAI
   Location: San Francisco, CA
   Skills: Machine learning, Data science, Python, R, SQL
   Contact: John Doe, john.doe@bayareaai.com

2. Role: Machine Learning Engineer
   Company: Data Innovators
   Location: San Francisco, CA
   Skills: Machine learning, Deep learning, Python, TensorFlow, PyTorch
   Contact: Sarah Smith, sarah.smith@datainnovators.com

3. Role: Data Analyst - AI Projects
   Company: Insight Analytics
   Location: San Francisco, CA
   Skills: Data analysis, AI projects, Tableau, Python, SQL
   Contact: Emily Johnson, emily.johnson@insightanalytics.com

4. Role: Data Scientist - NLP
   Company: Semantic Insights
   Location: San Francisco, CA
   Skills: Natural Language Processing, Machine learning, Python, NLTK
   Contact: Mark Thompson, mark.thompson@semanticinsights.com

5. Role: AI

In [22]:
response2 = retrieval_chain.invoke({
"chat_history":chat_history,
"input":"I am skilled in Java and Python with 8 years of experience. Looking for a job in Mumbai Location. Can you provide me some options?"
})
print(response2["answer"])

Based on your skills in Java and Python with 8 years of experience and your preference for a job in Mumbai, here are some opportunities that might interest you:

1. **Job Role:** Data Engineer (Python or Java, ETL, Hadoop, Spark)
   - **Location:** Mumbai, India
   - **Work Mode:** Onsite
   - **Duration:** Full-time
   - **Job Description:** Seeking data engineers with experience in Python, Java, ETL, Hadoop, and Spark to work on data pipelines in a cloud environment.
   
2. **Job Role:** Java Developer
   - **Location:** Mumbai, India
   - **Work Mode:** Remote
   - **Duration:** Contract
   - **Job Description:** Looking for Java developers with strong backend development skills for a remote project.
   
3. **Job Role:** Python Developer (Django)
   - **Location:** Mumbai, India
   - **Work Mode:** Onsite
   - **Duration:** Permanent
   - **Job Description:** Hiring Python developers with Django framework experience for web development projects.
   
Feel free to reach out to me if y

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

def chat(question, history):
    ai_message = retrieval_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.extend([HumanMessage(content=question), ai_message["answer"]])
    return ai_message['answer']

In [ ]:
import gradio as gr

gradio_interface = gr.ChatInterface(chat).launch()

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52c82cf4011814ca08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
